In [14]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from random import uniform, randint

# Crear la sesión de Spark
spark = SparkSession.builder.appName("RepresentatividadSQL").getOrCreate()

# Generar datos ficticios para los sensores
data = []
for i in range(100):
    sensor_id = f"{i+1}"
    fecha = datetime.now() - timedelta(days=i)
    temperatura = round(uniform(15.0, 30.0), 2) # Temperatura entre 15 y 30 grados
    nivel_agua = round(uniform(5.0, 20.0), 2) # Nivel de agua entre 5 y 20 metros
    caudal = randint(100, 500) # Caudal entre 100 y 500 litros por segundo
    presion = randint(900, 1500) # Presión entre 900 y 1500 Pascales
    estado = "Activo" if randint(0, 1) == 0 else "Inactivo" # Estado aleatorio

    data.append((sensor_id, fecha, temperatura, nivel_agua, caudal, presion, estado))

# Definir los nombres de las columnas
columns = ["Sensor_ID", "Fecha", "Temperatura", "Nivel_Agua", "Caudal", "Presion", "Estado"]

# Crear el DataFrame
df_sensores = spark.createDataFrame(data, columns)

# Mostrar el DataFrame
df_sensores.show(5)


+---------+--------------------+-----------+----------+------+-------+--------+
|Sensor_ID|               Fecha|Temperatura|Nivel_Agua|Caudal|Presion|  Estado|
+---------+--------------------+-----------+----------+------+-------+--------+
|        1|2025-08-28 02:29:...|      23.21|      6.73|   338|   1313|  Activo|
|        2|2025-08-27 02:29:...|      27.06|     16.15|   183|   1231|  Activo|
|        3|2025-08-26 02:29:...|      17.01|     17.82|   408|   1021|Inactivo|
|        4|2025-08-25 02:29:...|      27.32|     10.35|   127|   1144|  Activo|
|        5|2025-08-24 02:29:...|      23.33|     17.94|   309|   1341|Inactivo|
+---------+--------------------+-----------+----------+------+-------+--------+
only showing top 5 rows



In [15]:
# Crear vista temporal
df_sensores.createOrReplaceTempView("sensores")

1. Consultar los sensores activos con nivel de agua > 10 metros

In [16]:
# Crear consulta
Consulta_Nivel_Agua = spark.sql("""
                                SELECT Sensor_ID, Nivel_Agua, Temperatura, Caudal, Presion
                                FROM sensores
                                WHERE Estado = 'Activo' AND Nivel_Agua > 10
                                ORDER BY Nivel_Agua DESC""")
Consulta_Nivel_Agua.show()

+---------+----------+-----------+------+-------+
|Sensor_ID|Nivel_Agua|Temperatura|Caudal|Presion|
+---------+----------+-----------+------+-------+
|       85|     19.83|      26.67|   268|    967|
|       65|     19.08|      21.43|   325|   1384|
|       79|     19.08|      28.92|   148|   1014|
|       89|     18.81|      29.86|   417|   1336|
|       56|     17.96|      29.72|   321|   1429|
|       11|     17.85|      16.13|   472|   1027|
|       97|     17.02|      19.37|   245|   1144|
|       58|     16.27|      15.84|   337|   1301|
|        2|     16.15|      27.06|   183|   1231|
|       73|     15.87|      21.78|   129|   1155|
|       76|     15.29|      23.09|   357|   1300|
|       75|     14.83|      16.26|   174|   1249|
|       96|     14.79|      19.58|   249|   1005|
|       18|     14.26|      19.04|   190|   1310|
|       71|     14.07|      21.59|   353|   1464|
|       51|      14.0|      28.12|   499|   1066|
|       21|     13.53|      23.82|   304|   1473|


2. Obtener el promedio de la temperatura de todos los sensores

In [17]:
# Crear consulta
Consulta_Promedio_Temperatura = spark.sql("""
                                          SELECT ROUND(AVG(Temperatura), 2) as Temperatura_Promedio
                                          FROM sensores """)
Consulta_Promedio_Temperatura.show()

+--------------------+
|Temperatura_Promedio|
+--------------------+
|                23.1|
+--------------------+



3. Mostrar los sensores con el caudal más alto

In [18]:
# Crear consulta
Consulta_Max_Caudal = spark.sql("""
                                SELECT Sensor_ID, Caudal, Estado, Fecha
                                FROM sensores
                                ORDER BY Caudal DESC
                                LIMIT 10""")
Consulta_Max_Caudal.show()


+---------+------+--------+--------------------+
|Sensor_ID|Caudal|  Estado|               Fecha|
+---------+------+--------+--------------------+
|       51|   499|  Activo|2025-07-09 02:29:...|
|       17|   490|Inactivo|2025-08-12 02:29:...|
|       62|   484|Inactivo|2025-06-28 02:29:...|
|       26|   482|Inactivo|2025-08-03 02:29:...|
|       15|   481|Inactivo|2025-08-14 02:29:...|
|       11|   472|  Activo|2025-08-18 02:29:...|
|       40|   470|  Activo|2025-07-20 02:29:...|
|       61|   468|Inactivo|2025-06-29 02:29:...|
|       64|   468|Inactivo|2025-06-26 02:29:...|
|       23|   465|  Activo|2025-08-06 02:29:...|
+---------+------+--------+--------------------+



4. Filtrar sensores inactivos y mostrar fecha y presión

In [19]:
# Crear consulta
Consulta_Inactivos = spark.sql("""
                                SELECT Sensor_ID, Fecha, Presion, Temperatura, Nivel_Agua
                                FROM sensores
                                WHERE Estado = 'Inactivo'
                                ORDER BY Fecha DESC""")
Consulta_Inactivos.show()

+---------+--------------------+-------+-----------+----------+
|Sensor_ID|               Fecha|Presion|Temperatura|Nivel_Agua|
+---------+--------------------+-------+-----------+----------+
|        3|2025-08-26 02:29:...|   1021|      17.01|     17.82|
|        5|2025-08-24 02:29:...|   1341|      23.33|     17.94|
|        6|2025-08-23 02:29:...|   1170|      25.34|     12.73|
|        7|2025-08-22 02:29:...|   1310|      21.38|      7.77|
|        8|2025-08-21 02:29:...|   1187|      25.45|     13.32|
|       12|2025-08-17 02:29:...|    922|      25.57|      5.36|
|       13|2025-08-16 02:29:...|   1052|      28.14|     12.41|
|       15|2025-08-14 02:29:...|   1308|      27.07|     17.35|
|       16|2025-08-13 02:29:...|   1259|      22.01|     14.25|
|       17|2025-08-12 02:29:...|   1155|      27.08|     13.92|
|       19|2025-08-10 02:29:...|    926|      17.37|      5.38|
|       20|2025-08-09 02:29:...|   1305|      25.87|     13.69|
|       22|2025-08-07 02:29:...|    971|

5. Calcular el total de caudal acumulado por sensor

In [20]:
# Crear consulta
Consulta_Acumulado = spark.sql("""
                              SELECT Sensor_ID,
                              SUM(Caudal) as Caudal_Acumulado
                              FROM sensores
                              GROUP BY Sensor_ID
                              ORDER BY Caudal_Acumulado DESC""")
Consulta_Acumulado.show()

+---------+----------------+
|Sensor_ID|Caudal_Acumulado|
+---------+----------------+
|       51|             499|
|       17|             490|
|       62|             484|
|       26|             482|
|       15|             481|
|       11|             472|
|       40|             470|
|       64|             468|
|       61|             468|
|       23|             465|
|       74|             462|
|       35|             461|
|       33|             461|
|       86|             461|
|       67|             460|
|        6|             453|
|       22|             448|
|       70|             445|
|       43|             444|
|       55|             440|
+---------+----------------+
only showing top 20 rows

